# Research 06: Gradio HR Executive Interface

In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import os
import sys
import json
import gradio as gr
import pypdfium2 as pdfium
from dotenv import load_dotenv
from pathlib import Path
from PIL import Image
import google.generativeai as genai
import pandas as pd

In [3]:
load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

In [4]:
class HR_Executive:
    def __init__(self, model_n, instruction, api_key):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel(
            model_n,
            generation_config={"response_mime_type": "application/json"},
            system_instruction=instruction, 
        )
        
    
    def executive(self, inputs:list):
        response = self.model.generate_content(inputs)
        candidate = json.loads(response.text)
        return candidate

In [5]:
tech_hr_system_instruction = """
You are an experienced Technical Human Resource Manager. Your task is to review the provided resume/CV against the technical job description. 

Evaluate the candidate for the job based on the provided resume/CV. And extract the candidate contact details from resume/CV also, so that further HR executive can contact to candidate. Shortlist the candidate based on provided ats score.

Your evaluation should include:
1. A percentage match between the resume and the job description.
2. Candidate details, like Name, contact, email.
3. Final thoughts on the candidate's suitability for the position.
4. Shortlisted status[True or False]. True, if ats score satisfy the given ats score else False.

Give output in below JSON schema only:
candidate_evaluation = {
    "details": {
        "name": str,
        "contact": str,
        "email": str,
        "ats_score": int,
        "final_thoughts":str
    },
    "shortlist": boolean
}
"""

nontech_hr_system_instruction = """
You are an experienced Non-Technical Human Resource Manager. Your task is to review the provided resume/CV against the Non-Technical job description. 

Evaluate the candidate for the job based on the provided resume/CV. And extract the candidate contact details from resume/CV also, so that further HR executive can contact to candidate. Shortlist the candidate based on provided ats score.

Your evaluation should include:
1. A percentage match between the resume and the job description.
2. Candidate details, like Name, contact, email.
3. Final thoughts on the candidate's suitability for the position.
4. Shortlisted status[True or False]. True, if ats score satisfy the given ats score else False.

Give output in below JSON schema only:
candidate_evaluation = {
    "details": {
        "name": str,
        "contact": str,
        "email": str,
        "ats_score": int,
        "final_thoughts":str
    },
    "shortlist": boolean
}
"""

In [6]:
tech_hr = HR_Executive('gemini-1.5-flash', tech_hr_system_instruction, GEMINI_API_KEY)
non_tech_hr = HR_Executive('gemini-1.5-flash', nontech_hr_system_instruction, GEMINI_API_KEY)

In [7]:
def pdf_2_images(pdf_path):
    pages = pdfium.PdfDocument(pdf_path)

    images = []
    for i, page in enumerate(pages):
        image = page.render(scale=4).to_pil()
        images.append(image)
    return images

In [11]:
def process_inputs(hr, jd, resumes, ats_score):
    shortlisted_candidates = pd.DataFrame(columns=["name", "contact", "email", "ats_score", "final_thoughts"])

    prompt = f"Shortlist the candidate with minimum {ats_score} ats score."
    resume_imgs = None
    for resume in resumes:
        resume_imgs = pdf_2_images(resume)
    
        if hr == "Technical HR Executive":
            executive_response = tech_hr.executive([jd, *resume_imgs, prompt])
            if executive_response.get("shortlist", False):        
                shortlisted_candidates = shortlisted_candidates._append(executive_response.get("details"), ignore_index=True)
    
        if hr == "Non-Technical HR Executive":
            executive_response = tech_hr.executive([jd, *resume_imgs, prompt])
            if executive_response.get("shortlist", False):        
                shortlisted_candidates = shortlisted_candidates._append(executive_response.get("details"), ignore_index=True)
    
    return shortlisted_candidates

In [12]:
with gr.Blocks() as app:
    gr.Markdown("# Your Technical/Non-Technical HR Executive")
    hr = gr.Dropdown(choices=["Technical HR Executive", "Non-Technical HR Executive"], label="Select HR Executive")
    with gr.Row():
        with gr.Column(scale=3):
            jd = gr.Textbox(placeholder="Job Description here...", label="Job Description")
        with gr.Column():
            resumes = gr.File(label="Resume/CV", file_types=["file"], file_count='multiple')
            ats_score = gr.Slider(label="ATS Score")
            shortlist_btn = gr.Button("Shortlist")
    output = gr.Dataframe(label="Shortlisted Candidates")

    shortlist_btn.click(process_inputs, inputs=[hr, jd, resumes, ats_score], outputs=output)
app.launch()

Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
